In [ ]:
import datetime as datetime
import pandas as pd
import numpy as np
import csv 
import re
import matplotlib.pyplot as plt
from IPython.display import Image

In [ ]:
import pyspark

try: 
    type(sc)
except NameError:
    sc = pyspark.SparkContext('local[*]')

In [ ]:
df = pd.read_csv('Sources/fiuba_6_avisos_detalle_concat.csv').drop(['Unnamed: 0'],axis=1)
cluster = pd.read_csv('Sources/idaviso-cluster.csv').drop('Unnamed: 0',axis=1)
features = pd.read_csv('Sources/mas_features.csv').drop('Unnamed: 0',axis=1)
tuples = [tuple(x) for x in df.values]
df.head()

In [ ]:
for x in tuples:
    x = tuple(x)

In [ ]:
avisos = sc.parallelize(tuples)

In [ ]:
avisos = avisos.map(lambda x: (x[0],x[1],x[2],str.upper(re.sub("<.*?>", " ", x[3])),str.upper(re.sub("<.*?>", " ", x[4])),x[5],x[6],x[7],x[8],x[9],x[10],x[11])).map(lambda x: (x[0],x[1],x[2],str(re.sub("\xa0", " ", x[3])),str(re.sub("\xa0", " ", x[4])),x[5],x[6],x[7],x[8],x[9],x[10],x[11]))

In [ ]:
import re, string, unicodedata
def removePunctuation(text):
    text=str(text).lower().strip(' ')
    text=unicodedata.normalize('NFD', text).encode('ascii','ignore')
    text=text.decode('utf-8')
    text=re.sub(r'[^a-zA-Z0-9 ]', '', text)   
    return text

In [ ]:
def palabrasLargas(texto):
    lista = texto.split(' ') 
    lista = [x for x in lista if not len(x)>17]
    return " ".join(lista)

In [ ]:
avisos = avisos.map((lambda x: (x[0],x[1],x[2],removePunctuation(x[3]),removePunctuation(x[4]),x[5],x[6],x[7],x[8],x[9],x[10],x[11]))).map((lambda x: (x[0],x[1],x[2],palabrasLargas(x[3]),x[4],x[5],x[6],x[7],x[8], x[9],x[10],x[11]))).collect()

In [ ]:
avisos = pd.DataFrame.from_records(avisos).drop([0], axis=1)

In [ ]:
for x in avisos.columns:  
    avisos = avisos.rename(columns = {x:df.columns[x-1]})
avisos.head()
avisos = avisos.drop(['idpais','nombre_zona','mapacalle', 'ciudad'], axis=1)
avisos.head()

In [ ]:
avisos = avisos.merge(cluster,on='idaviso',how='left')
avisos.head()

In [ ]:
avisos = avisos.merge(features, on='idaviso',how='left')
avisos.head()

In [ ]:
avisos.to_csv("Sources/fiuba_6_avisos_detalle_limpio.csv")